In [1]:
import math
import itertools
import pandas

In [2]:
enum = { "nada": 0, "básico": 1, "médio": 2, "avançado": 3 }
inverse_enum = { 0: "nada", 1: "básico", 2: "médio", 3: "avançado" }
r = itertools.combinations_with_replacement([
    enum["nada"],
    enum["básico"],
    enum["médio"],
    enum["avançado"]
], 3)
r = list(r)

In [3]:
x = [enum["nada"],
    enum["básico"],
    enum["médio"],
    enum["avançado"]]


r = [p for p in itertools.product(x, repeat=3)]

r = list(r)

In [4]:
def format_result(i, sim):
    return {
        "Excel": inverse_enum[i[0]],
        "PPT": inverse_enum[i[1]],
        "Word": inverse_enum[i[2]],
        "Similaridade": sim
    }

## Sem match | Pesos iguais

$$S_{i} = \dfrac{\sum_{k = 0}^N \frac{r_{ik}}{|L| - 1}\frac{1}{C}^{r_{ik} - (|L| - 1)}}                                  {\sum_{k = 0}^N
			\frac{1}{C}^{r_{ik} - (|L| - 1)}}$$

$$ \frac{|L| - 1}{|L| - 2} - 1 < C < \frac{|L| - 1}{|L| - 2} $$

In [6]:
def no_match_equal_weights(i, C = [1.4, 1.4, 1.4], inverse = False):
    numerator = 0
    denominator = 0
    
    for _i,  _c in list(zip(i, C)):
        _c = 1/_c
        denominator += _c**(_i-3)
        numerator += (_i/3)*_c**(_i-3)        
        
    return format_result(i, numerator/denominator)

## Sem match | Pesos diferentes

$$S_{ij} = \dfrac{\sum_{k = 0}^N \frac{r_{ik}}{|L| -1 } \cdot (1 + \frac{r_{jk}}{|L| - 1}) \cdot \frac{1}{C}^{r_{ik} - ({|L| -1 })}}                                  {\sum_{k = 0}^N (1 + \frac{r_{jk}}{|L| -1 })  \cdot  \frac{1}{C}^{r_{ik} - ({|L| -1 })}}$$

$$ \frac{|L| - 1}{|L| - 2} - 1 < C < \frac{|L| - 1}{|L| - 2} $$

In [7]:
def no_match_diff_weights(i, j = [3, 3, 3], C = [1.4, 1.4, 1.4]):
    numerator = 0
    denominator = 0
    
    for _i, _j, _c in list(zip(i, j, C)):
        _c = 1/_c
        denominator += ((_j/3)+1)*_c**(_i-3)
        numerator += ((_i/3)*((_j/3)+1))*_c**(_i-3)        
        
    return format_result(i, numerator/denominator)

## Com match | Pesos diferentes

$$S_{ij} = \dfrac{\sum_{k = 0}^N \frac{r_{ik} - r_{jk}}{|L| -1 } \cdot (1 + \frac{r_{jk}}{|L| -1 }) \cdot C^{r_{ik} - ({|L| -1 })}}                                  {\sum_{k = 0}^N (1 + \frac{r_{jk}}{|L| -1 })  \cdot  C^{r_{ik} - ({|L| -1 })}}$$

$$ \frac{|L| - 1}{|L| - 2} - 1 < C < \frac{|L| - 1}{|L| - 2} $$

$$ C = \begin{cases} C, & \mbox{if } r_{ik} \leq r_{jk} \\ \frac{1}{C}, & \mbox{if } r_{ik} > r_{jk} \end{cases} $$

In [5]:
def with_match_diff_weights(i, j = [3, 3, 3], C = [1.4, 1.4, 1.4]):
    numerator = 0
    denominator = 0
    
    for _i, _j, _c in list(zip(i, j, C)):
        _c = 1/_c
        denominator +=  ((_j/3)+1)*_c**(_i-3)
        numerator += ((_i - _j)/3)*((_j/3)+1)*_c**(_i-3)

    return format_result(i, numerator/denominator)

# Generalizações


### Sem match | Pesos iguais

1. Para qualquer valor de $ i_k $, $i_k + 1$ sempre terá um valor de similaridade maior que $i_k$;
2. Dado um valor de $i_k$ menor que $|L|- 1$, $C + 1$ sempre resultará em um valor de similaridade maior que $C$;
3. Independente do valor de $C$, se $i$ for igual à $|L| - 1$, todos os valores de similaridade serão iguais;
4. $S_i \in	[0, 1]$

### Sem match | Pesos diferentes

1. Para um mesmo valor de $ i_k $ entre objetos, um objeto com $ j_k + 1 $ resultará em um valor de similaridade menor que $j_k$. *Motivo: ao aumentar o peso de $j_k$, conhecimentos menores são mais "penalizados" e portanto resultam em similaridades menores*;
2. Para um mesmo valor de $j_k$ entre objetos, um objeto com $i_k + 1$ resultará em um valor de similaridade maior do que um objeto com $i_k$;
3. Para um objeto com $i_k$, $j_k$ e $C$ e outro com $i_k$, $j_k$ e $C + 1$, o primeiro terá um valor de similaridade menor que o segundo;
4. Dados objetos com os mesmos valores de $i_k$, para um objeto com todos $j_k$ iguais, um objeto com todos $j_k + 1$ resultará em um valor de similaridade igual ao objeto anterior;
5. Para um objeto com todos $i_k$ iguais, o valor de similaridade será o mesmo independente do $j_k$ e $C$;
6. $S_{ij} \in [0, 1]$

### Com match | Pesos diferentes

*Herda as generalizações 1, 2 e 3 de Sem match | Pesos diferentes*

1. Dado que $i_k$ de um objeto é igual à $j_k$, o valor de similaridade será $0$, independente do valor de $C$
2. Dado que $i_k$ de um objeto é menor que $j_k$, o valor de similaridade será no intervalo $]-1, 0[$
3. Dado que $i_k$ de um objeto é maior que $j_k$, o valor de similaridade será no intervalo $]0, 1[$
4. $S_{ij} \in	 [-1,1]$, porém ocupará um subintervalo de módulo igual à $1$, por exemplo, de $-0.55$ a $0.45$ e $-0.9$ a $0.1$

------------------------

### Pontos importantes

**Por que em Com match | Pesos diferentes a constante $C$ é invertido quando $i_k$ é maior que $j_k$?**

Por causa da possibilidade de um valor negativo. Antes de chegar na inversão, a seguinte tabela definia o comportamento do mérito e o valor de `C`.

Supondo que $i_k = 1$

- | Match negativo | Match positivo | $$ \Rightarrow $$ | Match negativo | Match positivo 
--- | --- | --- | --- | --- | --- 
C grande | $$ -(5^{-2}) = -0.04 $$ | $$ 5^{-2} = 0.04 $$ | $$ \Rightarrow $$ | bastante mérito | pouco mérito
C pequeno | $$ -(0.7^{-2}) = -2.04 $$ | $$ 0.7^{-2} = 2.04 $$ | $$ \Rightarrow $$ | pouco mérito | bastante mérito


Se o match é negativo e o valor de $C$ for alto, então o mérito é alto também, porém isso não ocorre quando o match é positivo e sim o oposto, por isso o valor de $C$ é invertido quando $i_k$ é maior que $j_k$. Dessa forma, o usuário não precisa inverter manualmente. Assim chega-se na seguinte tabela:

- | Match negativo | Match positivo | $$ \Rightarrow $$ | Match negativo | Match positivo 
--- | --- | --- | --- | --- | --- 
C grande | $$ -(5^{-2}) = -0.04 $$ | $$ \frac{1}{5}^{-2} = 25 $$ | $$ \Rightarrow $$ | bastante mérito | bastante mérito
C pequeno | $$ -(0.7^{-2}) = -2.04 $$ | $$ \frac{1}{0.7}^{-2} = 0.49 $$ | $$ \Rightarrow $$ | pouco mérito | pouco mérito

E isso nos leva a rever as medidas de sem match, em específico o valor de C. Ambas as medidas sem match são como que fossem uma com match porém somente com o match positivo, portanto, se a medida que as engloba define que para um match positivo, C = 1/C, então elas devem definir C = 1/C também, ao invés de ser simplesmente C.



## Mérito: C

### Interpretação


- C grande: deêm ênfase maior para os valores pequenos -> valorizem mais os pequenos => saber menos afeta mais
- C pequeno: deêm ênfase menor para os valores pequenos -> valorizem menos os pequenos => saber menos afeta menos


Um valor menor que 1 de C atribuí um peso maior à conhecimentos menores, o que é contraintuívo, porém não se pode esquecer que a media ponderada é tirada, assim essa aparente controversa. Exemplo:

```py
>>> [no_match_equal_weights([0, 0], [1.4, 1.4]),
 no_match_equal_weights([1, 0], [1.4, 1.4]),
 no_match_equal_weights([2, 0], [1.4, 1.4]),
 no_match_equal_weights([3, 0], [1.4, 1.4])]
[{'Similaridade': 0.0},
 {'Similaridade': 0.1388888888888889},
 {'Similaridade': 0.2252252252252252},
 {'Similaridade': 0.2670940170940171}]
 
>>> [no_match_equal_weights([0, 0], [1.4, 0.5]),
 no_match_equal_weights([1, 0], [1.4, 0.5]),
 no_match_equal_weights([2, 0], [1.4, 0.5]),
 no_match_equal_weights([3, 0], [1.4, 0.5])]
[{'Similaridade': 0.0},
 {'Similaridade': 0.3133493205435651},
 {'Similaridade': 0.6120218579234973},
 {'Similaridade': 0.8888888888888888}]
```

A diferença da similaridade dos níveis do conhecimento quando o C é igual à 0.5 segue uma função linear ao invés de exponencial, como é no caso de C = 1.4

### Estudo 

### Com match e pesos diferentes vs Sem match e pesos diferentes

Entretanto, ao executar mais cenários, o comportamento de $C$ se torna instável e um intervalo exato não consegue ser definido. Por exemplo, dado a medidade **com match** em que $j = [1, 1, 1]$ e varia entre $i = [3, 3, 1]$ e $i = [3, 3, 2]$, o valor de $C$ para o último atributo não pode ser $1.4$, e sim $<= 1.3$

- Último C igual à 1.4

```py
>>> with_match_diff_weights(i = [3, 3, 2], j = [1, 1, 1], C = [0.6, 0.6, 1.4])
{ 'Similaridade': 0.5294117647058824 }

>>> with_match_diff_weights(i = [3, 3, 1], j = [1, 1, 1], C = [0.6, 0.6, 1.4])
{ 'Similaridade': 0.5311653116531165 }
```

Resulta em uma similariade maior para [3, 3, 1] do que [3, 3, 2], o que errado, a ordem deveria ser o contrário.


- Último C igual à 1.3

```py
>>> with_match_diff_weights(i = [3, 3, 2], j = [1, 1, 1], C = [0.6, 0.6, 1.3])
{ 'Similaridade': 0.5353535353535352 }

>>> with_match_diff_weights(i = [3, 3, 1], j = [1, 1, 1], C = [0.6, 0.6, 1.3])
{ 'Similaridade': 0.5144596651445966 }
```

Resulta em uma similariade menor para [3, 3, 1] do que [3, 3, 2], o que é correto.

Quando a medida **sem match e com pesos diferentes** é utilizada nesse cenários, o valor do último $C$ pode sim assumir 1.4, não é necessário que seja menor ou igual à 1.3


- Último C igual à 1.4

```py
>>> no_match_diff_weights(i = [3, 3, 2], j = [1, 1, 1], C = [0.6, 0.6, 1.4])
{ 'Similaridade': 0.8627450980392156 }

>>> no_match_diff_weights(i = [3, 3, 1], j = [1, 1, 1], C = [0.6, 0.6, 1.4])
{ 'Similaridade': 0.6700336700336701 }
```

Resulta em uma similariade menor para [3, 3, 1] do que [3, 3, 2], o que é correto.


- Último C igual à 1.3

```py
>>> no_match_diff_weights(i = [3, 3, 2], j = [1, 1, 1], C = [0.6, 0.6, 1.3])
{ 'Similaridade': 0.8686868686868685 }

>>> with_match_diff_weights(i = [3, 3, 1], j = [1, 1, 1], C = [0.6, 0.6, 1.3])
{ 'Similaridade': 0.6946702800361337 }
```

Resulta em uma similariade menor para [3, 3, 1] do que [3, 3, 2], o que é correto.


## Cenários

antes de inverter => peso dos conhecimentos baixos ficava muito baixo para o C grande (1.5) e portanto não eram valorizados e assim não impactavam a medida final, dado o peso pequeno. Com o C invertido, um C grande significa que os conhecimentos pequenos vão ter pesos maiores e assim vão ter mais impacto,

- Ah, saber pouco em PPT não afeta muito o candidado (C pequeno), porém, saber pouco em Word afeta bastante (C grande).

In [14]:
with_match_diff_weights(i = [3, 3, 2], j = [1, 1, 1], C = [0.6, 0.6, 1.3])

{'Excel': 'avançado',
 'PPT': 'avançado',
 'Word': 'médio',
 'Similaridade': 0.5353535353535352}

In [15]:
with_match_diff_weights(i = [3, 3, 1], j = [1, 1, 1], C = [0.6, 0.6, 1.3])

{'Excel': 'avançado',
 'PPT': 'avançado',
 'Word': 'básico',
 'Similaridade': 0.36133694670280037}

In [13]:
with_match_diff_weights([1, 2, 3], [1, 2, 3], [1.3, 1.3, 0.5])

{'Excel': 'básico', 'PPT': 'médio', 'Word': 'avançado', 'Similaridade': 0.0}

In [12]:
with_match_diff_weights([3, 2 , 1], [1, 2, 3], [1.3, 1.3, 0.5])

{'Excel': 'avançado',
 'PPT': 'médio',
 'Word': 'básico',
 'Similaridade': 0.1388888888888889}

# Tests

In [59]:
result = sorted(list(map(lambda i: no_match_equal_weights(i, [0.7, 0.7, 0.7]), list(r))), key=lambda x: x["Similaridade"])
df = pandas.DataFrame(result)
print(df.to_latex())

\begin{tabular}{llllr}
\toprule
{} &     Excel &       PPT &      Word &  Similaridade \\
\midrule
0  &      nada &      nada &      nada &      0.000000 \\
1  &      nada &      nada &    básico &      0.138889 \\
2  &      nada &    básico &      nada &      0.138889 \\
3  &    básico &      nada &      nada &      0.138889 \\
4  &      nada &    básico &    básico &      0.246914 \\
5  &    básico &      nada &    básico &      0.246914 \\
6  &    básico &    básico &      nada &      0.246914 \\
7  &    básico &    básico &    básico &      0.333333 \\
8  &      nada &      nada &     médio &      0.336700 \\
9  &      nada &     médio &      nada &      0.336700 \\
10 &     médio &      nada &      nada &      0.336700 \\
11 &      nada &    básico &     médio &      0.410959 \\
12 &      nada &     médio &    básico &      0.410959 \\
13 &    básico &      nada &     médio &      0.410959 \\
14 &    básico &     médio &      nada &      0.410959 \\
15 &     médio &      nada &   

In [55]:
result = sorted(list(map(lambda i: no_match_diff_weights(i, [3,3,3], [0.8, 0.8, 1.5]), list(r))), key=lambda x: x["Similaridade"])
df = pandas.DataFrame(result)
print(df.to_latex())

\begin{tabular}{llllr}
\toprule
{} &     Excel &       PPT &      Word &  Similaridade \\
\midrule
0  &      nada &      nada &      nada &      0.000000 \\
1  &      nada &    básico &      nada &      0.047125 \\
2  &    básico &      nada &      nada &      0.047125 \\
3  &    básico &    básico &      nada &      0.091658 \\
4  &      nada &     médio &      nada &      0.113790 \\
5  &     médio &      nada &      nada &      0.113790 \\
6  &    básico &     médio &      nada &      0.155071 \\
7  &     médio &    básico &      nada &      0.155071 \\
8  &      nada &  avançado &      nada &      0.204625 \\
9  &  avançado &      nada &      nada &      0.204625 \\
10 &     médio &     médio &      nada &      0.214405 \\
11 &      nada &      nada &    básico &      0.229078 \\
12 &    básico &  avançado &      nada &      0.241941 \\
13 &  avançado &    básico &      nada &      0.241941 \\
14 &      nada &    básico &    básico &      0.283167 \\
15 &    básico &      nada &   

In [263]:
result = sorted(list(map(lambda i: with_match_diff_weights(i, [3,3,3], [0.6, 0.6, 1.5]), list(r))), key=lambda x: x["Similaridade"])
df = pandas.DataFrame(result)
print(df.to_latex())

\begin{tabular}{llllr}
\toprule
{} &     Excel &       PPT &      Word &  Similaridade \\
\midrule
0  &      nada &      nada &      nada &     -1.000000 \\
1  &      nada &    básico &      nada &     -0.969628 \\
2  &    básico &      nada &      nada &     -0.969628 \\
3  &    básico &    básico &      nada &     -0.941392 \\
4  &      nada &     médio &      nada &     -0.904557 \\
5  &     médio &      nada &      nada &     -0.904557 \\
6  &    básico &     médio &      nada &     -0.880046 \\
7  &     médio &    básico &      nada &     -0.880046 \\
8  &     médio &     médio &      nada &     -0.825137 \\
9  &      nada &  avançado &      nada &     -0.782183 \\
10 &  avançado &      nada &      nada &     -0.782183 \\
11 &    básico &  avançado &      nada &     -0.763464 \\
12 &  avançado &    básico &      nada &     -0.763464 \\
13 &      nada &      nada &    básico &     -0.720358 \\
14 &     médio &  avançado &      nada &     -0.718593 \\
15 &  avançado &     médio &   

In [40]:
result = list(map(lambda i: with_match_diff_weights(i, [2, 2, 2], [1.3, 1.3, 1.3]), list(r)))

for i, v in enumerate(result):
    result[i]['idx'] = i
    
#result = sorted(result, key=lambda x: x["Similaridade"])

In [28]:
pandas.set_option('display.max_rows', None)

In [41]:
df = pandas.DataFrame(result)
df

,Excel,PPT,Word,Similaridade,idx
0,nada,nada,nada,-0.666667,0
1,nada,nada,básico,-0.574074,1
2,nada,nada,médio,-0.514460,2
3,nada,nada,avançado,-0.481275,3
4,nada,básico,nada,-0.574074,4
5,nada,básico,básico,-0.464646,5
6,nada,básico,médio,-0.390977,6
7,nada,básico,avançado,-0.346770,7
8,nada,médio,nada,-0.514460,8
9,nada,médio,básico,-0.390977,9
